# train


In [2]:
import math
import os

from absl import app
from absl import flags
from absl import logging
import numpy as np
import tensorflow as tf

def get_data():

  x_values = np.random.uniform(low=0, high=2 * math.pi,size=1000).astype(np.float32)

  np.random.shuffle(x_values)

  y_values = np.sin(x_values).astype(np.float32)

  return (x_values, y_values)

In [3]:
def create_model() -> tf.keras.Model:
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(1, )))

  model.add(tf.keras.layers.Dense(16, activation='relu'))

  model.add(tf.keras.layers.Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mae'])

  return model

In [4]:
def convert_tflite_model(model):

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()
  return tflite_model


In [5]:
def save_tflite_model(tflite_model, save_dir, model_name):

  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
  save_path = os.path.join(save_dir, model_name)
  with open(save_path, "wb") as f:
    f.write(tflite_model)
  logging.info("Tflite model saved to %s", save_dir)

In [6]:
def train_model(epochs, x_values, y_values):

  model = create_model()
  model.fit(x_values,
            y_values,
            epochs=epochs,
            validation_split=0.2,
            batch_size=64,
            verbose=2)

  return model

In [7]:
x_values, y_values = get_data()
trained_model = train_model(100, x_values, y_values)

  # Convert and save the model to .tflite
tflite_model = convert_tflite_model(trained_model)
save_tflite_model(tflite_model,
                  "./",
                    model_name="hello_world_float.tflite")

Epoch 1/100
13/13 - 1s - loss: 0.4228 - mae: 0.5730 - val_loss: 0.3998 - val_mae: 0.5545 - 1s/epoch - 79ms/step
Epoch 2/100
13/13 - 0s - loss: 0.3885 - mae: 0.5345 - val_loss: 0.3756 - val_mae: 0.5349 - 53ms/epoch - 4ms/step
Epoch 3/100
13/13 - 0s - loss: 0.3637 - mae: 0.5193 - val_loss: 0.3572 - val_mae: 0.5271 - 55ms/epoch - 4ms/step
Epoch 4/100
13/13 - 0s - loss: 0.3417 - mae: 0.5052 - val_loss: 0.3354 - val_mae: 0.5086 - 80ms/epoch - 6ms/step
Epoch 5/100
13/13 - 0s - loss: 0.3210 - mae: 0.4894 - val_loss: 0.3156 - val_mae: 0.4944 - 55ms/epoch - 4ms/step
Epoch 6/100
13/13 - 0s - loss: 0.3005 - mae: 0.4751 - val_loss: 0.2963 - val_mae: 0.4790 - 67ms/epoch - 5ms/step
Epoch 7/100
13/13 - 0s - loss: 0.2795 - mae: 0.4579 - val_loss: 0.2780 - val_mae: 0.4656 - 98ms/epoch - 8ms/step
Epoch 8/100
13/13 - 0s - loss: 0.2613 - mae: 0.4441 - val_loss: 0.2589 - val_mae: 0.4479 - 83ms/epoch - 6ms/step
Epoch 9/100
13/13 - 0s - loss: 0.2420 - mae: 0.4260 - val_loss: 0.2439 - val_mae: 0.4383 - 93ms/e

# hello world lab

In [ ]:
!git clone https://github.com/Yui-Arthur/tinyML_TFLM_lab.git

In [23]:
%cd tinyML_TFLM_lab/hello_world_lab/hello_world

/content/hello_world


## run script

In [ ]:
# setup the tflm lib
!bash setup.sh
# change flatbuffers header to fb_*
!bash preprocess.sh

## export .tflite model to .cc

In [8]:
!xxd -i /content/hello_world_float.tflite > model.cc
!echo -ne "#include \"model_data.h\"\nalignas(8)\n" > model_data.cc
!cat model.cc >> model_data.cc
!sed -i -E 's/(unsigned\s.*\s).*(_len|\[\])/const \1model\2/g' model_data.cc

# compile

In [12]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3 << '2'
!sudo apt install python3-pip
!sudo apt-get install python3.7-distutils
!python -m pip install --upgrade --force-reinstall pip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minima

In [13]:
!python -V
!pip -V

Python 3.7.17
pip 23.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
!sudo apt install git mercurial libusb-1.0-0-dev
!pip install mbed-cli
# install python dependency

!wget https://raw.githubusercontent.com/ARMmbed/mbed-os/master/requirements.txt
!sed -i 's/psutil==5.6.7/ /' requirements.txt
!pip install -r requirements.txt

In [ ]:
!wget -O gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2 https://developer.arm.com/-/media/Files/downloads/gnu-rm/9-2020q2/gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2?revision=05382cca-1721-44e1-ae19-1e7c3dc96118&rev=05382cca172144e1ae191e7c3dc96118&hash=FDE675133A099796BD1507A3FF215AC4
!sudo tar -jxvf ./gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2 -C /usr/local/

In [15]:
!pip install psutil
!pip install Jinja2
!pip install jsonschema
!pip install Pillow
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.6/283.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 34.4 MB/s eta 0:00:00


In [29]:
!mbed config -G GCC_ARM_PATH /usr/local/gcc-arm-none-eabi-9-2020-q2-update/bin
!mbed config -G TOOLCHAIN GCC_ARM
!mbed add mbed-os
!mbed deploy

[mbed] /usr/local/gcc-arm-none-eabi-9-2020-q2-update/bin now set as global GCC_ARM_PATH
[mbed] GCC_ARM now set as global TOOLCHAIN
[mbed] Working path "/content/hello_world" (program)
[mbed] WARNING: Program "hello_world" in "/content/hello_world" does not use source control management.
       To fix this you should use "mbed new ." in the root of your program.
---
[mbed] ERROR: Directory "/content/hello_world/mbed-os" is not empty. Please ensure that the destination folder is empty.
---
[mbed] Working path "/content/hello_world" (program)
[mbed] WARNING: Program "hello_world" in "/content/hello_world" does not use source control management.
       To fix this you should use "mbed new ." in the root of your program.
---
[mbed] Updating library "mbed-os" to rev #7c7d20da6527
[mbed] Updating library "tflite-micro/tensorflow/lite/micro/tools/make/downloads/cmsis_nn/Tests/UnitTest/Mbed/mbed-os" to rev #6a244d7adffc (tags: mbed-os-5.15.5, mbed-os-5.15.5-rc1)


In [26]:
!mbed add /content/hello_world/libtensorflow-microlite.a

[mbed] Working path "/content/hello_world" (program)
[mbed] WARNING: Program "hello_world" in "/content/hello_world" does not use source control management.
       To fix this you should use "mbed new ." in the root of your program.
---
[mbed] Adding library "libtensorflow-microlite.a" from "/content/hello_world/libtensorflow-microlite.a" at latest revision in the current branch
[mbed] ERROR: Unable to clone repository (/content/hello_world/libtensorflow-microlite.a/)
---


In [45]:
!sudo mbed compile

[mbed] Working path "/content/hello_world" (program)
Building project hello_world (NUCLEO_F767ZI, GCC_ARM)
Scan: hello_world
Compile [100.0%]: hello_world.cc
Link: hello_world
Elf2Bin: hello_world
| Module           |     .text |    .data |     .bss |
|------------------|-----------|----------|----------|
| [fill]           |  114(-20) |    4(+0) |   12(+0) |
| [lib]/c.a        | 12468(+0) | 2472(+0) |   89(+0) |
| [lib]/gcc.a      |  3232(+0) |    0(+0) |    0(+0) |
| [lib]/m.a        |  3564(+0) |    0(+0) |    0(+0) |
| [lib]/misc       |   188(+0) |    4(+0) |   28(+0) |
| [lib]/nosys.a    |    32(+0) |    0(+0) |    0(+0) |
| hello_world.o    |  564(+20) |    0(+0) |    0(+0) |
| mbed-os/drivers  |   102(+0) |    0(+0) |    0(+0) |
| mbed-os/hal      |   886(+0) |    4(+0) |   58(+0) |
| mbed-os/platform |  4620(+0) |  260(+0) |  295(+0) |
| mbed-os/targets  |  8860(+0) |    8(+0) | 1254(+0) |
| model_data.o     |     0(+0) |    0(+0) |    0(+0) |
| signal/micro     |    50(+0) | 